# Packages

In [17]:
using BenchmarkTools
using Cuba
using LorentzVectors
using Plots
using DelimitedFiles
using DataInterpolations
using Polylogarithms

# Tolerance integration

In [52]:
tol = 1e-5

1.0e-5

# Constants of nature and Dynkin indices

In [53]:
############  Particle physics and cosmo constant ###############
MP = 2.435*1e18              #[GeV]     Reduced Planck mass
T0 = 2.72548*8.6173e-5*1e-9  #[GeV]     CMB temperature
Tmax = 1e15                  #[GeV]     Maximal temperature of the plasma (should change that, maybe as an input parameter)
g_Tmax = 178                 #          # of entropic dofs at the maximal temperature of the plasma
g_Tew = 106.75               #          # of entropic dofs at the electroweak scale


########### Dynkin indices SO(10) ######################
#to change later, I just don't know for now (should be an array)
T_fermion = 1       
T_scalar = 1
T_Ad = 1

########### N_leptons and N_species ######################
#Based on the Dynkin indices
N_leptons = 1    #to change, I just don' know for now
N_species = (sum(T_fermion)+sum(T_scalar))/2

1.0

# Defining the Bose-Einstein distribution

In [54]:
function nb(x)
    return 1/(exp(x)-1)
end

nb (generic function with 1 method)

# Defining the $\hat{\eta}$ functions

In [57]:
############## Define the L_i and M_i functions, necessary for the eta functions ############

function L1p(x,y)
    return log((1-exp(-(x+y)/2))/(1-exp((x-y)/2)))
end
function L1m(x,y)
    return log((1+exp(-(x+y)/2))/(1+exp((x-y)/2)))
end

function L2p(x,y)
    return real(polylog(2,exp((x-y)/2))-polylog(2,exp(-(x+y)/2)))
end
function L2m(x,y)
    return real(polylog(2,-exp((x-y)/2))-polylog(2,-exp(-(x+y)/2)))
end

function L3p(x,y)
    return real(polylog(3,exp((x-y)/2))-polylog(3,exp(-(x+y)/2)))
end
function L3m(x,y)
    return real(polylog(3,-exp((x-y)/2))-polylog(3,-exp(-(x+y)/2)))
end




function M1p(x,y)
    return log((1-exp(-(x-y)/2))/(1-exp(-(x+y)/2)))
end
function M1m(x,y)
    return log((1+exp(-(x-y)/2))/(1+exp(-(x+y)/2)))
end

function M2p(x,y)
    return real(polylog(2,exp(-(x+y)/2))+polylog(2,exp(-(x-y)/2)))
end
function M2m(x,y)
    return real(polylog(2,-exp(-(x+y)/2))+polylog(2,-exp(-(x-y)/2)))
end

function M3p(x,y)
    return real(polylog(3,exp(-(x+y)/2))-polylog(3,exp(-(x-y)/2)))
end
function M3m(x,y)
    return real(polylog(3,-exp(-(x+y)/2))-polylog(3,-exp(-(x-y)/2)))
end


############## Define I,J,K,L's, which are lin. comb. of eta functions ##################

function Ip(k)
    xmin = -10
    function boundary1(x)
        return abs(x)
    end
    function boundary2(x)
        return 2*k-x
    end
    function integrand(xx,ff)
        x,y=xx
        varchangex = (k-xmin)*x+xmin
        varchangey = (boundary2(varchangex)-boundary1(varchangex))*y+boundary1(varchangex)
        ff[1] = (k-xmin)*(boundary2(varchangex)-boundary1(varchangex))*((1+nb(varchangex)+nb(k-varchangex))*(-2/3*L1p(varchangex,varchangey)+(varchangey^2-3*(varchangex-2*k)^2)*(12*L3p(varchangex,varchangey)+6*varchangey*L2p(varchangex,varchangey)+varchangey^2*L1p(varchangex,varchangey))/6/varchangey^4)+4*k^2*pi^2/varchangey^4)*(varchangey^2-varchangex^2)
    end
    result = cuhre(integrand,atol = tol, rtol = tol).integral[1]
    return nb(k)/(256*pi^3*k)*result
end


function Im(k)
    xmin = -10
    function boundary1(x)
        return abs(x)
    end
    function boundary2(x)
        return 2*k-x
    end
    function integrand(xx,ff)
        x,y=xx
        varchangex = (k-xmin)*x+xmin
        varchangey = (boundary2(varchangex)-boundary1(varchangex))*y+boundary1(varchangex)
        ff[1] = (k-xmin)*(boundary2(varchangex)-boundary1(varchangex))*((1+nb(varchangex)+nb(k-varchangex))*(-2/3*L1m(varchangex,varchangey)+(varchangey^2-3*(varchangex-2*k)^2)*(12*L3m(varchangex,varchangey)+6*varchangey*L2m(varchangex,varchangey)+varchangey^2*L1m(varchangex,varchangey))/6/varchangey^4)-2*k^2*pi^2/varchangey^4)*(varchangey^2-varchangex^2)
    end
    result = cuhre(integrand,atol = tol, rtol = tol).integral[1]
    return nb(k)/(256*pi^3*k)*result
end


function Jp(k)
    xmax = 10
    function boundary1(x)
        return abs(2*k-x)
    end
    function boundary2(x)
        return x
    end
    function integrand(xx,ff)
        x,y=xx
        varchangex = (xmax-k)*x+xmax
        varchangey = (boundary2(varchangex)-boundary1(varchangex))*y+boundary1(varchangex)
        ff[1] = (xmax-k)*(boundary2(varchangex)-boundary1(varchangex))*(nb(varchangex-k)-nb(varchangex))*(varchangey^2-varchangex^2)*(1/3*(2*M1p(varchangex,varchangey)-varchangey)-(varchangey^2-3*(varchangex-2*k)^2)*(12*M3p(varchangex,varchangey)+6*varchangey*M2p(varchangex,varchangey)+varchangey^2*M1p(varchangex,varchangey))/6/varchangey^4)
    end
    result = cuhre(integrand,atol = tol, rtol = tol).integral[1]
    return nb(k)/(256*pi^3*k)*result
end


function Jm(k)
    xmax = 10
    function boundary1(x)
        return abs(2*k-x)
    end
    function boundary2(x)
        return x
    end
    function integrand(xx,ff)
        x,y=xx
        varchangex = (xmax-k)*x+xmax
        varchangey = (boundary2(varchangex)-boundary1(varchangex))*y+boundary1(varchangex)
        ff[1] = (xmax-k)*(boundary2(varchangex)-boundary1(varchangex))*(nb(varchangex-k)-nb(varchangex))*(varchangey^2-varchangex^2)*(1/3*(2*M1m(varchangex,varchangey)-varchangey)-(varchangey^2-3*(varchangex-2*k)^2)*(12*M3m(varchangex,varchangey)+6*varchangey*M2m(varchangex,varchangey)+varchangey^2*M1m(varchangex,varchangey))/6/varchangey^4)
    end
    result = cuhre(integrand,atol = tol, rtol = tol).integral[1]
    return nb(k)/(256*pi^3*k)*result
end
     

function K(k)
    xmin = -10
    function boundary1(x)
        return abs(x)
    end
    function boundary2(x)
        return 2*k-x
    end
    function integrand(xx,ff)
        x,y=xx
        varchangex = (k-xmin)*x+xmin
        varchangey = (boundary2(varchangex)-boundary1(varchangex))*y+boundary1(varchangex)
        ff[1] = (k-xmin)*(boundary2(varchangex)-boundary1(varchangex))*(1+nb(varchangex)+nb(k-varchangex))*L1m(varchangex,varchangey)*(varchangey^2-varchangex^2)
    end
    result = cuhre(integrand,atol = tol, rtol = tol).integral[1]
    return -nb(k)/(256*pi^3*k)*result
end


function L(k)
    xmax = 10
    function boundary1(x)
        return abs(2*k-x)
    end
    function boundary2(x)
        return x
    end
    function integrand(xx,ff)
        x,y=xx
        varchangex = (xmax-k)*x+xmax
        varchangey = (boundary2(varchangex)-boundary1(varchangex))*y+boundary1(varchangex)
        ff[1] = (xmax-k)*(boundary2(varchangex)-boundary1(varchangex))*(nb(varchangex-k)-nb(varchangex))*(2*M1m(varchangex,varchangey)-varchangey)*(varchangey^2-varchangex^2)
    end
    result = cuhre(integrand,atol = tol, rtol = tol).integral[1]
    return nb(k)/(256*pi^3*k)*result
end







################ Define the eta functions ###################

function etagg(k)
    return -4*(Ip(k)+Jp(k))
end

function etasg(k)
    return -(Ip(k)+Jp(k))
end

function etasf(k)
    return 4*K(k)+2*L(k)
end

function etafg(k)
    return 4*(Im(k)+Jm(k))
end


etafg (generic function with 1 method)

In [60]:
start_time=time()
println(etasf(1))
println('\n',time()-start_time, " sec")

0.0007063042782790742

0.14344286918640137 sec
